# Dowenload era5 land using era5cli

This notebook shows how to download [era5 data](https://climate.copernicus.eu/) using [era5cli](https://era5cli.readthedocs.io/en/latest/index.html).  

In [1]:
# import python libraries
from netCDF4 import Dataset, num2date
from pathlib import Path

## Run these two cells below only once

In [ ]:
# Create a key ascii file
!echo "url: https://cds.climate.copernicus.eu/api/v2" > ~/.cdsapirc 

# Replace UID with your user ID and KEY with your API key
!echo "key: UID:KEY" >> ~/.cdsapirc

In [ ]:
# install era5cli
!pip install era5cli

## Settings

In [2]:
# path to forcing data, for example on CRIB data are located at:
forcing_path = "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/Plumber2_data/"

## Test run using --dryrun

In [3]:
forcing_name = "AU-ASM_2011-2017_OzFlux_Met.nc"

# read data
filename = Path(forcing_path, forcing_name) # care for windows/unix paths
nc_fid = Dataset(filename, mode='r')

# get time information
nctime = nc_fid.variables['time']
ncdate = num2date(nctime[:],units= nctime.units, calendar=nctime.calendar)

startyear = ncdate[0].year
endyear = startyear
startmonth = ncdate[0].month
startday = ncdate[0].day
starthour = ncdate[0].hour

lat = float(nc_fid.variables['latitude'][:].flatten()[0])
lon = float(nc_fid.variables['longitude'][:].flatten()[0])

# create area, note that era5 resolution is 0.1
lat_max = lat + 0.001
lat_min = lat - 0.001
lon_max = lon + 0.001
lon_min = lon - 0.001

# create prefix for file name
station_name = forcing_name.split('_')[0]
timestamp = ncdate[0].strftime('%Y%m%d_%H')
file_name = f"{station_name}_{timestamp}"

In [4]:
print(startyear, endyear, startmonth, startday, starthour, lat, lon)

2011 2011 1 1 0 -22.283000946044922 133.24899291992188


In [5]:
print(lat_max, lon_min, lat_min, lon_max)

-22.28200094604492 133.24799291992187 -22.284000946044923 133.24999291992188


## Run era5cli 
The cell below only shows the results of the command-line. It does **not** download any data. 

In [15]:
# Run era5cli for variables soil_temperature and volumetric_soil_water at four levels and skin_temperature
!era5cli hourly --variables soil_temperature_level_1 soil_temperature_level_2 soil_temperature_level_3 soil_temperature_level_4 \
volumetric_soil_water_layer_1 volumetric_soil_water_layer_2 volumetric_soil_water_layer_3 volumetric_soil_water_layer_4 skin_temperature \
--startyear {startyear} --endyear {endyear} --land \
--area {lat_max} {lon_min} {lat_min} {lon_max} \
--months {startmonth} --days {startday} --hours {starthour} --outputprefix {file_name} --dryrun --merge

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188]

## Download data

The cell below downloads data. You can specify the name of forcing files or download data at all 170 sites. 

In [ ]:
## specify the forcing file names or set fullrun = True
forcing_filenames_list = ["AR-SLu_2010-2010_FLUXNET2015_Met.nc", "AU-ASM_2011-2017_OzFlux_Met.nc", ]

## if you want to download all 170 sites, change fullrun = False to fullrun = True
fullrun = False
if fullrun:
    forcing_filenames_list = [file.name for file in Path(forcing_path).iterdir()]
    
for forcing_name in forcing_filenames_list:
    # read data
    filename = Path(forcing_path, forcing_name) # care for windows/unix paths
    nc_fid = Dataset(filename, mode='r')

    # get time information
    nctime = nc_fid.variables['time']
    ncdate = num2date(nctime[:],units= nctime.units, calendar=nctime.calendar)

    startyear = ncdate[0].year
    endyear = startyear
    startmonth = ncdate[0].month
    startday = ncdate[0].day
    starthour = ncdate[0].hour

    lat = float(nc_fid.variables['latitude'][:].flatten()[0])
    lon = float(nc_fid.variables['longitude'][:].flatten()[0])

    # create area, note that era5 resolution is 0.1
    lat_max = lat + 0.001
    lat_min = lat - 0.001
    lon_max = lon + 0.001
    lon_min = lon - 0.001

    # create prefix for file name
    station_name = forcing_name.split('_')[0]
    timestamp = ncdate[0].strftime('%Y%m%d_%H')
    file_name = f"{station_name}_{timestamp}"

    # download data
    !era5cli hourly --variables soil_temperature_level_1 soil_temperature_level_2 soil_temperature_level_3 soil_temperature_level_4 \
    volumetric_soil_water_layer_1 volumetric_soil_water_layer_2 volumetric_soil_water_layer_3 volumetric_soil_water_layer_4 skin_temperature \
    --startyear {startyear} --endyear {endyear} --land \
    --area {lat_max} {lon_min} {lat_min} {lon_max} \
    --months {startmonth} --days {startday} --hours {starthour} --outputprefix {file_name} --merge

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188] rounded down to two decimals.

NB: coordinates [-22.28200094604492, 133.24799291992187, -22.284000946044923, 133.24999291992188]